# 행렬분해를 이용한 잠재 요인 협업 필터링 

In [2]:
# https://github.com/wikibook/ml-definitive-guide

In [1]:
#rmse 평가 함수
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [2]:
#SGD 기반의 행렬 분해 구현
#R = 원본 사용자-아이템 평점 행렬
#K = 잠재요인의 차원 수
#steps = SGD 반복 횟수
#learning_rate :  학습률
#r_lambda = L2 규제 계수


def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

In [3]:
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
ratings = ratings[['userId','movieId','rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId') #사용자-아이템 평점 행렬 생성

In [7]:
#title 칼럼을 얻기 위해 movies와 join
rating_movies = pd.merge(ratings, movies, on='movieId')
#colums = 'title' 로 pivot 수행
ratings_maric = rating_movies.pivot_table('rating', index='userId', columns='title')

In [9]:
#사용자-아이템 평점 행렬으 matrix_factorization() 함수를 이용하여 행렬 분해
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  2.699204301709659
### iteration step :  10  rmse :  0.7661956225249948
### iteration step :  20  rmse :  0.530932088589884
### iteration step :  30  rmse :  0.3861024370616224
### iteration step :  40  rmse :  0.3068932366504682
### iteration step :  50  rmse :  0.2614593544750243
### iteration step :  60  rmse :  0.23351685641956446
### iteration step :  70  rmse :  0.21500263987266544
### iteration step :  80  rmse :  0.20191325984631808
### iteration step :  90  rmse :  0.19217987643832224
### iteration step :  100  rmse :  0.18465739069584783
### iteration step :  110  rmse :  0.17866670395271386
### iteration step :  120  rmse :  0.17378142790193918
### iteration step :  130  rmse :  0.16972123965633512
### iteration step :  140  rmse :  0.1662946053696789
### iteration step :  150  rmse :  0.16336620703951366
### iteration step :  160  rmse :  0.1608375829201499
### iteration step :  170  rmse :  0.15863520128683947
### iteration step :  180  rmse

In [10]:
#예측 사용자-아이템 평점 행렬을 DataFrame으로 변경
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index = ratings_matrix.index, columns = ratings_matrix.columns)

ratings_pred_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.219413,2.608552,4.131222,2.905101,3.626638,4.181860,4.133656,3.840058,3.670304,4.751578,...,3.032766,2.538513,3.400288,3.424991,3.083075,3.524228,3.017725,3.048294,3.065846,3.528559
2,3.797129,3.520305,4.203124,2.001279,3.178498,4.553509,3.131503,3.552015,3.417139,4.965818,...,2.728220,2.349227,2.981027,3.038971,2.732721,3.118977,2.691300,2.611148,2.687410,3.257123
3,1.783638,1.091696,1.584180,0.879563,-0.255868,2.633172,1.469916,2.951577,2.267331,0.963066,...,1.802815,1.481549,2.018660,1.934994,1.838333,2.064976,1.720691,1.666882,1.636386,2.065915


In [11]:
#사용자가 이미 평점을 준 영화를 제외하고 추천할 수 있도록 평점을 주지 않은 영화를 리스트 객체로 반환
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [12]:
#사용자에게 영화 추천
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [14]:
#사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

#잠재 요인 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix,9,unseen_list, top_n=10)

#평점 데이터를 DataFrame으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, columns=['pred_score'])
recomm_movies

,pred_score
0,5.459654
1,5.290213
2,5.231366
3,5.204406
4,4.994479
5,4.956023
6,4.942915
7,4.922988
8,4.911752
9,4.901672
